# Analysis of extrinsic results

TODO: Summary of extrinsic 3a and 3b?
- everything in extrinsic summaries - read results, redo tables

TODO: Analysis of spatial autocorrelation of:

- differences in density
- matched/unmatched features
- connectivity
- components per grid cell?

TODO: Analysis of overlap between errors and differences.

TODO: Analysis of correlation between high/low quality areas and pop density

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
import json
import pandas as pd
import seaborn as sns
import plotly.express as px

## Differences per municipality

In [12]:
# Load results of municipal analysis

osm_muni = pd.read_csv("../results/osm_quality/muni_network_counts.csv",index_col=False)
ref_muni = pd.read_csv("../results/geodk_quality/muni_network_counts.csv",index_col=False)


In [13]:
osm_muni

,infrastructure_length,node_count,dangling_node_count,infra_km,navn,area_sqkm,infra_dens,pop,infra_pop,overshoots,undershoots,over_under,component_count
0,335114.384014,1238,751,335.114,Aabenraa,940.61,0.356,59002,5.679706,2.0,NaN,0.0,230
1,670714.668671,3708,1841,670.715,Aalborg,1138.00,0.589,222571,3.013488,2.0,8.0,10.0,444
2,937870.720797,5153,2391,937.871,Aarhus,471.20,1.990,361544,2.594072,7.0,14.0,21.0,430
3,110961.318630,509,196,110.961,Albertslund,23.44,4.734,27530,4.030548,2.0,1.0,3.0,30
4,157756.329035,1150,512,157.756,Allerød,67.43,2.340,26061,6.053336,2.0,5.0,7.0,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,453020.522640,2330,1264,453.021,Vejle,1061.61,0.427,120949,3.745554,1.0,1.0,2.0,285
94,295317.880306,766,384,295.318,Vesthimmerlands,768.04,0.385,36431,8.106228,NaN,NaN,0.0,95
95,453235.159585,2277,1147,453.235,Viborg,1421.26,0.319,97731,4.637577,5.0,NaN,0.0,298
96,180389.972247,602,324,180.390,Vordingborg,625.26,0.289,45441,3.969763,NaN,NaN,0.0,92


In [11]:
ref_muni

,infrastructure_length,node_count,dangling_node_count,infra_km,navn,area_sqkm,infra_dens,pop,infra_pop,overshoots,undershoots,over_under,component_count
0,193541.697984,1390,126,193.542,Aabenraa,940.61,0.206,59002,3.280262,NaN,6.0,0.0,51
1,218764.628637,1478,306,218.765,Aalborg,1138.00,0.192,222571,0.982900,NaN,12.0,0.0,102
2,240556.379891,2097,690,240.556,Aarhus,471.20,0.511,361544,0.665357,3.0,32.0,35.0,271
3,9893.782706,85,61,9.894,Albertslund,23.44,0.422,27530,0.359390,1.0,NaN,0.0,30
4,54391.846843,296,68,54.392,Allerød,67.43,0.807,26061,2.087103,1.0,4.0,5.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,203558.475958,1002,219,203.558,Vejle,1061.61,0.192,120949,1.683007,1.0,21.0,22.0,86
94,41079.393930,221,62,41.079,Vesthimmerlands,768.04,0.053,36431,1.127584,NaN,NaN,0.0,26
95,401855.089934,1199,256,401.855,Viborg,1421.26,0.283,97731,4.111848,1.0,4.0,5.0,112
96,75289.385711,363,109,75.289,Vordingborg,625.26,0.120,45441,1.656852,NaN,6.0,0.0,54


In [ ]:
# TODO; Make a new dataframe - all the differences

# TODO: Make a new dataframe - pct differences

# TODO: compare ranks?

# TODO: Plot correlation between diff